## EDA  🚡

In [ ]:
# Importing libraries 
import os
import cv2
import ast
import json
import subprocess
from glob import glob
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Video

In [ ]:
df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
# Checking for duplicacy 
duplicate = df.duplicated().sum()
print('No. of duplicates :', duplicate)
print("_______________________________")
# Checking for no.of annotations per file
for video_id in df['video_id'].unique():
    print(f'video_id: {video_id}')
    print(f'with (w) annotations: {sum(df[df["video_id"]==video_id]["annotations"]=="[]")}')
    print(f'without (w/o) annotations: {sum(df[df["video_id"]==video_id]["annotations"]!="[]")}\n')

In [ ]:
# Bounding box

df[df.annotations.str.len()>2]

## Visualizing the bbox on images ⭐⭐

In [ ]:
## Visualizing the bbox on images 
# changing the type of annotations from str to list
df['annotations'] = df['annotations'].apply(ast.literal_eval)
# add path of images, number of boxes , differences
input_path = '../input/tensorflow-great-barrier-reef/train_images/'
df['image_path'] = input_path + 'video_' + df['video_id'].astype(str) + '/' + df['video_frame'].astype(str) +'.jpg'
df['num_bboxes'] = df['annotations'].apply(lambda x: len(x))
df['diff_num_bboxes'] = df['num_bboxes'].diff().fillna(0).astype(int)
df.head()

**Plotting the boxes on image**

In [ ]:
"""Getting bounding box annotations"""
def get_bboxes(annotations):
    """
    annotations: list of annotations
    return: bboxes as [x_min, y_min, x_max, y_max]
    """
    if len(annotations) == 0:
        return []
    bboxes = pd.DataFrame(annotations, columns=['x', 'y', 'width', 'height']).astype(np.int32).values
    # [x_min, y_min , w, h] ->> [x_min, y_min , x_max, y_max]
    bboxes[:, 2] = bboxes[:, 0] + bboxes[:, 2]
    bboxes[:, 3] = bboxes[:, 1] + bboxes[:, 3]
    return bboxes

In [ ]:
"""plotting the image"""
def plot_img_with_bbox(img_path, annotations):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig , ax = plt.subplots(1,1, figsize=(16,10))
    if len(annotations) > 0:
        bboxes = get_bboxes(annotations)
        for i, box in enumerate(bboxes):
            cv2.rectangle(img,(box[0], box[1]),(box[2], box[3]), color = (255, 0, 0), thickness= 2)
            # numbering
            ax.text(box[0], box[1]-5, i+1, color='red')
    ax.set_axis_off()
    ax.imshow(img)

In [ ]:
sample_idx = 18
sample = df.iloc[sample_idx]
print(sample)

In [ ]:
img_path    = sample['image_path']
annotations = sample['annotations']
print('image_id:', sample['image_id'])
# plot image with bboxes
plot_img_with_bbox(img_path, annotations)

Work in progress !!!!